In [1]:

comp_table = {
    '0': '0101010', '1': '0111111', '-1': '0111010', 'D': '0001100',
    'A': '0110000', 'M': '1110000', '!D': '0001101', '!A': '0110001',
    '!M': '1110001', '-D': '0001111', '-A': '0110011', '-M': '1110011',
    'D+1': '0011111', 'A+1': '0110111', 'M+1': '1110111', 'D-1': '0001110',
    'A-1': '0110010', 'M-1': '1110010', 'D+A': '0000010', 'D+M': '1000010',
    'D-A': '0010011', 'D-M': '1010011', 'A-D': '0000111', 'M-D': '1000111',
    'D&A': '0000000', 'D&M': '1000000', 'D|A': '0010101', 'D|M': '1010101'
}

dest_table = {
    'null': '000',
    'M': '001',
    'D': '010',
    'MD': '011',
    'A': '100',
    'AM': '101',
    'AD': '110',
    'AMD': '111'
}

jump_table = {
    'null': '000',
    'JGT': '001',
    'JEQ': '010',
    'JGE': '011',
    'JLT': '100',
    'JNE': '101',
    'JLE': '110',
    'JMP': '111'
}

def c_instruction(line):
    ins = "111"

    def dest(line):
        if '=' in line:
            return line.split('=')[0].strip()  # splitting in two parts with '=' and taking the first part
        return 'null'  # if no '=' null

    def comp(line):
        s = line
        if '=' in s:
            s = s.split('=')[1].strip()  # takign the part after '='
        if ';' in s:
            return s.split(';')[0].strip() # taking the part before  ';'
        return s.strip()

    def jump(line):
        if ';' in line:
            return line.split(';')[1].strip() # taking teh part after ';'
        return 'null'

    try:
        ins += comp_table[comp(line)]   # computation
        dest_part = dest(line)
        ins += dest_table[dest_part]      # destination
        jump_part = jump(line)
        ins += jump_table[jump_part]     # jump
    except KeyError as e:
        raise KeyError(f"Error with key: {e}")

    return ins


In [2]:


class Parser:
    def hasMoreLines(self, file): #Checking if htere are more lines
        current_pos = file.tell()
        line = file.readline()
        file.seek(current_pos)
        return bool(line)

    def advance(self, file):
        return file.readline().strip() # Returning the line

    def instruction_type(self, line): # instruction type classification
        if line.startswith('@'):
            return 'A_INSTRUCTION'
        elif line.startswith('(') and line.endswith(')'):
            return 'L_INSTRUCTION'
        else:
            return 'C_INSTRUCTION'

    def symbol(self, line): # Extracting the symbol
        if line.startswith('@'):
            return line[1:]
        elif line.startswith('(') and line.endswith(')'):
            return line[1:-1]
        return None


In [8]:


symbols = {
    'SP': 0, 'LCL': 1, 'ARG': 2, 'THIS': 3, 'THAT': 4,
    'R0': 0, 'R1': 1, 'R2': 2, 'R3': 3, 'R4': 4, 'R5': 5, 'R6': 6, 'R7': 7,
    'R8': 8, 'R9': 9, 'R10': 10, 'R11': 11, 'R12': 12, 'R13': 13, 'R14': 14, 'R15': 15,
    'SCREEN': 16384, 'KBD': 24576
}

class SymbolTable:
    def __init__(self):
        self.table = symbols.copy()
        self.va = 16  # For variables address starts from 16

    def addEntry(self, symbol, address): # add new variable with its address in symbol table
        self.table[symbol] = address

    def contains(self, symbol):
        return symbol in self.table # checking if the symbol is there in symbol table

    def getAddress(self, symbol): # fetching the adress for the given symbol
        return self.table[symbol]


In [9]:


class Assembler:
    def __init__(self):
        self.symbol_table = SymbolTable()  # initialising symbol table
        self.output_lines = [] # output for storing the binary codes

    def first_pass(self, lines):   # first pass
        ia = 0 # assigning instrcution the value 0
        for line in lines:
            line = line.strip()  # getting the line
            if not line or line.startswith('//'): # checking if it's a comment or not to skip
                continue

            if line.startswith('(') and line.endswith(')'):  # checking for labels
                label = line[1:-1]   # extracting the labels
                self.symbol_table.addEntry(label, ia)  # adding a symbol corresponding to the label
            else:
                ia += 1  # number of instrucitons += 1

    def second_pass(self, lines): # second pass
        for line in lines:
            line = line.strip()
            if not line or line.startswith('//') or line.startswith('('):
                continue  # Skip comments, empty lines, or label declarations

            if line.startswith('@'):  # address instructiom
                symbol = line[1:] # getting the value
                if symbol.isdigit():  # checking if it is a number or variable
                    address = int(symbol)  # adress set to that number simply D=A
                else:
                    if not self.symbol_table.contains(symbol):  # if not a number checking if it exists in symbol table or not
                        self.symbol_table.addEntry(symbol, self.symbol_table.va)  # if not add hte address in the symbol table
                        self.symbol_table.va += 1  # symbol table adress for variable incrementation
                    address = self.symbol_table.getAddress(symbol)  # fetching the address of the variable
                binary_instruction = f'0{address:015b}' # converting the address to binary instruction
                self.output_lines.append(binary_instruction) # adding in output lines array

            else:  # C-instruction
                binary_instruction = c_instruction(line)  # Finding corresponding C-instruction binary
                self.output_lines.append(binary_instruction) # adding in output array

    def assemble(self, input_file_path, output_file_path):  #  assembling everything
        parser = Parser()  # setting up the parser
        with open(input_file_path, 'r') as file:  # opening the file
            lines = file.readlines()  # reading the lines

        # building the symbol table for the variables in the first pass
        self.first_pass(lines)

        # storing the binary conversion of the code in second pass
        self.second_pass(lines)

        # storing the output to the .hack file and printing it as well
        with open(output_file_path, 'w') as output_file:
            for line in self.output_lines:
                print(line)
                output_file.write(line + '\n')


In [11]:

def main(input_file_path):
    output_file_path = '/content/result.hack'  # making a hack file with name as result.hack
    assembler = Assembler()  # initialising the assembler
    assembler.assemble(input_file_path, output_file_path)  # printing the binary format

if __name__ == "__main__":
    input_file_path = '/content/Q8.asm'  # Specify your input assembly file path here I used colab so it is like this
    main(input_file_path)


0000000000010000
1110101010001000
0110000000000000
1111110000010000
0000000000000000
1110001100001000
0000000000010001
1110001100000101
0000000000010000
1111110000010000
0000000000010000
1111110010001000
0100000000000000
1110000010100000
1110101010001000
0000000000000010
1110101010000111
0000000000010000
1111110000010000
0000000000010000
1111110111001000
0100000000000000
1110000010100000
1110111010001000
0000000000000010
1110101010000111
